In [ ]:
# dependencies
import pandas as pd
import numpy as np
import altair as alt
from sklearn.preprocessing import OrdinalEncoder as OE
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.feature_selection import SelectKBest, chi2 , f_classif
from datetime import datetime
import ipywidgets as widgets
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
out = widgets.Output()

Description of the data: 

This files shows the top 180 recommended datasets on medium for each of the following searches:Data Science,Machine Learning,Artificial Intelligence,AI as of 05-06-21.

post_time:Describes the date at which the article was uploaded.

post_name:Name of the Article.

post_author:This shows the name of author or Anonymous if not Given,

post_publication:This shows the publication of the article if any independent if not.

post_claps:This gives the claps that the article recieved in Thousands.

post_responses = The number of comments that the article got.

post_tag 1-4 : The 4 tags used on the article "did'nt use" if no tag was used"

ranking:Its position on the search list.

Searched: This shows the Query that was Searched.

In [ ]:
input_csv = "../input/mediumsearchdataset/Train.csv"
df = pd.read_csv(input_csv)
df

In [ ]:
df.keys()

In [ ]:
categorical_data = ['post_tag_1', 'post_tag_2','post_tag_3', 'post_tag_4','Searched','post_publication','post_author']
numerical_data = ['post_claps', 'post_responses']
text_data = ['post_name']
time_data = ['post_time']

For this Eda I'll create a Arbitary label of the bottom 20 rankings , to check the difference between them. These are the top 200 articles in Medium Search So they're all good

In [ ]:
def Label_Maker(ranking):
    if ranking >= 160:
        return "Bad"
    else :
        return "Good"

In [ ]:
df["Label"] = df['ranking'].apply(Label_Maker)

In [ ]:
df

In [ ]:
features = [  'post_author', 'post_publication',
       'post_claps', 'post_responses', 'post_tag_1', 'post_tag_2',
       'post_tag_3', 'post_tag_4', 'ranking', 'Searched']
label = "Label"
categorical_data = ['post_tag_1', 'post_tag_2','post_tag_3', 'post_tag_4','Searched','post_publication','post_author']

In [ ]:
X = df[features]
y = df[label]

In [ ]:
le = LE()
for x in categorical_data:
    X[x] = le.fit_transform(X[x])

In [ ]:
select_features = SelectKBest( f_classif,k = 5).fit(X,y)
selected_features_df = pd.DataFrame({
    "Features" : features,
    "Scores" : select_features.scores_
})
selected_features_df

In [ ]:
alt.Chart(selected_features_df).mark_bar().encode(
    x = "Features",
    y = "Scores"
)


Since the Arbitary Label was made using  "ranking" it's not surprising that it has the highest feature importance.
Pretty straight forward with post_claps and post_responses as well the articles are ordered in the order of decending Claps however it is surprising that post_responses has a high Score than 

post_tags also scored very high. Surprisingly post_tag_3 had a higher score than most other tags will have to look into that.

In [ ]:
def plot_maker(cat_feature):
    plot = alt.Chart(df).mark_bar().encode(
        y = cat_feature,
        x = "count()"
    ).interactive()
    return plot

In [ ]:
plot_maker('post_tag_1')

In [ ]:
plot_maker('post_tag_2')

In [ ]:
plot_maker('post_tag_3')

In [ ]:
plot_maker('post_tag_4')

In [ ]:
plot_maker('post_publication')

In [ ]:
plot_maker('post_author')

Insights:-
from
Not surprisingly most authors did'nt use the tags (since most medium viewers get their articles recommended or it could be a problem with the dataset also the tags feature was added a bit later so some of these articles could be a bit old)
Apart from that Data Science,Machine Learning,Artificial Intelligence,Data Visualization tags were most used not surprising as the search was about these topics.
Data ,Data Analysis,Education,interview,Programming,Statistics,Deeplearning,How To & Career,Python,Startup were also highly used
.Design was a tag I didnt expect to be used so much, to be fair I don't know what it's refering to.

From Publications unsurprisingly TowardsDataScience crushed the competition with Nightangle,Hackernoon.com following behind.
Sadly independent articles were less than a Hundred.(Sad Because I'm a independent Author.)

From the Authors Elijah Meeks was at the top. From the top 180 most authors have only written 1 or 2 Articles which made it to the top 100. This is probably due to the Medium Recommendation Algorithm , if everyone got recommended articles from the Top authors than the Algorithm would be biased to the Authors who Started Early.


In [ ]:
def histogram_maker(num_category,max_):
    plot = alt.Chart(df).mark_bar().encode(
        alt.Y(num_category, bin= {"binned": False, "step": 0.05 , "extent" : [0,max_]}),
        x='count()'
    )
    return(plot)

In [ ]:
numerical_data = [('post_claps',25), ('post_responses',97)]
for x,max_ in numerical_data:
    display(histogram_maker(x,max_))
print("Standard Deviation Of Post_responces = ",df['post_responses'].std())
print("Standard Deviation Of Post_claps = ",df['post_claps'].std())

Data is as expected
In post_claps most Claps fall in the Range between between 0- 5K
In post_responces most responses fell in the range between 0-12 

Post_responses  has a very high Standard Deviation Which is probably due to top articles having a large number of responses. Which Skews the mean.

In [ ]:
def year_extr(yr):
    return yr.strftime("%Y")

In [ ]:
df['post_time'] = pd.to_datetime(df['post_time'])
df['year'] = df['post_time'].apply(year_extr)

In [ ]:
histogram_maker("year",2022)

Most Articles are from 2018 which makes sense articles since eariler uploads will have  higher claps and views. Medium Algorithm Likely recommends Newer Articles more As can be seen in the graph. The Amount of articles in 2021 is less as when this Data was acquired the we are about halfway through the year

In [ ]:
wordcloud = WordCloud(background_color = "black",height = 800 ,width = 1500).generate(''.join(df["post_name"].tolist()))
plt.figure(figsize=(30,30),facecolor='k')
plt.imshow(wordcloud)

A word colud for Visualization purposes

next we'll check to see which search queries had the highest number of claps. (probably data science but we'll see who gets second)

In [ ]:
search = df['Searched'].unique()
search_clap_sum = []
for x in search:
    sum_ = df[df["Searched"] == x]["post_claps"].sum()
    search_clap_sum.append(sum_)
searches_df = pd.DataFrame({
    "Search" : search,
    "Search_sum_of_claps" : search_clap_sum
})

In [ ]:
searches_df

In [ ]:
alt.Chart(searches_df).mark_bar().encode(
    x = "Search",
    y = "Search_sum_of_claps"
)

Machine Learning And Data Science has the highest number of claps. Followed by Artificial Intelligence  and AI.

In [ ]:
def tag_count(tag):
    count = []
    unique = df[tag].unique()
    for x in unique:
        count.append(df[df[tag] == x][tag].count())
    posts_df = pd.DataFrame({
    "Posts" : unique,
    "Count" : count
})
    print(tag, "Top 10")
    display(posts_df.sort_values(by = "Count" , ascending = False).head(10))

In [ ]:
post_data = ['post_tag_1', 'post_tag_2','post_tag_3', 'post_tag_4']

for x in post_data:
    tag_count(x)